In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)

In [2]:
# Generate plots inline
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Support for FTrace events parsing and visualization
import trappy

# Support to configure and run RTApp based workloads
from wlgen import RTA, Ramp, Step, Pulse, Periodic

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Let's use the local host as a target
te = TestEnv(
    # Setup a target configuration
    target_conf = {
    
    # Define the kind of target platform to use for the experiments
    "platform"    : 'android',  # Linux system, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    
    # Preload settings for a specific target
    "board"       : 'juno2',   # load JUNO specific settings, e.g.
                              # - HWMON based energy sampling
                              # - Juno energy model
                              # valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board
                              # - oak   - Mediatek MT63xx based target

    # Define devlib module to load
    "modules"     : [
        'bl',           # enable big.LITTLE support
        'cpufreq',       # enable CPUFreq support
        'hwmon'
    ],

    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
             "sched_switch",
             "cpu_frequency"
         ],
         "buffsize" : 10240
    },

    # Account to access the remote target
    "host"        : '10.1.210.46',
    #"username"    : 'root',
    #"password"    : '',

    # Comment the following line to force rt-app calibration on your target
    #"rtapp-calib" : {
    #    '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
    #}
    "rtapp-calib" : { "0": 323, "1": 124, "2": 125, "3": 319, "4": 315, "5": 315 }

}

)

11:54:37  INFO    :         Target - Using base path: /data/work/juno/lisa
11:54:37  INFO    :         Target - Loading custom (inline) target configuration
11:54:37  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq', 'hwmon']
11:54:37  INFO    :         Target - Connecting Android target [10.1.210.46:5555]
11:54:37  INFO    :         Target - Connection settings:
11:54:37  INFO    :         Target -    {'device': '10.1.210.46:5555'}
11:54:38  INFO    :         Target - Initializing target workdir:
11:54:38  INFO    :         Target -    /data/local/tmp/devlib-target
11:54:40  INFO    :         Target - Topology:
11:54:40  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
11:54:41  INFO    :       Platform - Loading default EM:
11:54:41  INFO    :       Platform -    /data/work/juno/lisa/libs/utils/platforms/juno2.json
11:54:41  INFO    :         FTrace - Enabled tracepoints:
11:54:41  INFO    :         FTrace -   sched_switch
11:54:41  INFO    :         FTrace -  

# Create a new RTA workload generator object

The wlgen::RTA class is a workload generator which exposes an API to configure
RTApp based workload as well as to execute them on a target.

In [4]:
# Create a new RTApp workload generator
rtapp = RTA(
    
    target=te.target, # Target execution on the local machine
    
    name='example', # This is the name of the JSON configuration file reporting
                    # the generated RTApp configuration
    
    #calibration={0: 10, 1: 11, 2: 12, 3: 13} # These are a set of fake
    #                                         # calibration values
)

11:54:48  INFO    :          WlGen - Setup new workload example


The function here, build_perf_benchmark_rtapp, demonstrates how we can build an rt-app job description which uses a controller task to unblock a number of child tasks, which then each perform a set amount of work. This is intended to simulate performance benchmarks. Many aspects of the test can be configured via parameters.

In [5]:
def build_perf_benchmark_rtapp(run_duration_ms, calibration_cpu_name, num_tasks, iterations, logdir="/data/local/tmp", file_name="perfbench.json"):
    # static content
    json_content = { 
        'global': {
            'calibration': calibration_cpu_name,
            'default_policy': "SCHED_OTHER",
            'duration': -1,
            'logdir': logdir
        },
        'tasks': {
           'controller': {
                'loop': iterations+2, 
                'phases': {
                    'init_delay': {
                        'sleep': run_duration_ms*4
                    }
                }
            }
        }
    }
    # dynamic content (number of tasks)
    for cpu in range(0,num_tasks):
        bench_thread_name = "bench{}".format(cpu)
        # describe the worker thread
        json_content['tasks'][bench_thread_name] = {
                'loop': iterations, 
                'phases': {
                    'go': {
                        'run': run_duration_ms
                    }, 
                    'wait': {
                        'suspend': bench_thread_name
                    }
                }
            }
        # hook it to the controller
        json_content['tasks']['controller']['phases']["trigger{}".format(cpu)] = { 'resume': bench_thread_name }
    
    with open(file_name, 'w') as outfile:
            json.dump(json_content, outfile,
                    sort_keys=True, indent=4, separators=(',', ': '))
    return (file_name, json_content)
    

In [6]:
run_duration_ms = 500000
calibration = 'CPU1'
num_tasks = 6
iterations = 8

(filename, inline_config)=build_perf_benchmark_rtapp(run_duration_ms, calibration, num_tasks, iterations)

# Configure this RTApp instance to:
name=rtapp.conf(
    # 1. generate a "profile based" set of tasks
    #kind='profile',
    kind='custom',
    duration=-1,    
    # 2. define the "profile" of each task
    # to use inline job description:
    params=inline_config,
# to use filename instead:
#    params="{}".format(filename),

    # 3. use this folder for task logfiles
    run_dir='/data/local/tmp'
)

logging.info('Generated RTApp JSON:')
print json.dumps(inline_config, indent=4, sort_keys=True)


11:54:50  INFO    : Generated RTApp JSON:


{
    "global": {
        "calibration": "CPU1", 
        "default_policy": "SCHED_OTHER", 
        "duration": -1, 
        "logdir": "/data/local/tmp"
    }, 
    "tasks": {
        "bench0": {
            "loop": 8, 
            "phases": {
                "go": {
                    "run": 500000
                }, 
                "wait": {
                    "suspend": "bench0"
                }
            }
        }, 
        "bench1": {
            "loop": 8, 
            "phases": {
                "go": {
                    "run": 500000
                }, 
                "wait": {
                    "suspend": "bench1"
                }
            }
        }, 
        "bench2": {
            "loop": 8, 
            "phases": {
                "go": {
                    "run": 500000
                }, 
                "wait": {
                    "suspend": "bench2"
                }
            }
        }, 
        "bench3": {
            "loop": 8, 
            

In [7]:
def configure_target(is_big_little=0, initial_task_util=0, num_of_cpus=6, governor="sched", disabled_idle_states=[]):
    # device paths
    isBigLittle='/proc/sys/kernel/sched_is_big_little'
    initialTaskUtil='/proc/sys/kernel/sched_initial_task_util'
    cpuFreqGovernor='/sys/devices/system/cpu/cpu{}/cpufreq/scaling_governor'
    cpuIdleState='/sys/devices/system/cpu/cpu{}/cpuidle/state{}/disable'
    
    # Configure 'is_big_little' flag
    te.target.execute("echo {} > {}".format(is_big_little, isBigLittle))
    # Configure 'initial_task_util' flag
    te.target.execute("echo {} > {}".format(initial_task_util, initialTaskUtil))
    # set cpufreq governors
    te.target.execute("echo {} > {}".format(governor, cpuFreqGovernor.format(0)))
    te.target.execute("echo {} > {}".format(governor, cpuFreqGovernor.format(1)))
    # configure idle states
    idle_states=[ 0,1,2 ]
    if len(disabled_idle_states):
        for state in disabled_idle_states:
            idle_states.remove(state)
    
    for cpu in range(0, num_of_cpus):
        # disable states
        if len(disabled_idle_states):
            for idle_state in disabled_idle_states:
                te.target.execute("echo 0 > {}".format(cpuIdleState.format(cpu, idle_state)))
        #enable states
        if len(idle_states):
            for idle_state in idle_states:
                te.target.execute("echo 1 > {}".format(cpuIdleState.format(cpu, idle_state)))
    # report
    #print "sched_is_big_little:     {}".format(te.target.read_value(isBigLittle))
    #print "sched_initial_task_util: {}".format(te.target.read_value(initialTaskUtil))
    #print "Little Cluster Governor: {}".format(te.target.read_value(cpuFreqGovernor.format(0)))
    #print "Big Cluster Governor:    {}".format(te.target.read_value(cpuFreqGovernor.format(1)))
    #idle_states=[ 0,1,2 ]
    #for cpu in range(0, num_of_cpus):
    #    print "Cpu {} Idle States:".format(cpu)
    #    for state in idle_states:
    #        val = te.target.read_value(cpuIdleState.format(cpu, state))
    #        result='disabled'
    #        if val > 0:
    #            result='enabled'
    #        print "  State {} : {}".format(state, result)

In [8]:
configure_target(governor='sched')

logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

11:55:03  INFO    : #### Setup FTrace
11:55:07  INFO    : #### Start energy sampling
11:55:07  INFO    : #### Start RTApp execution
11:55:07  INFO    :          WlGen - Workload execution START:
11:55:07  INFO    :          WlGen -    /data/local/tmp/bin/rt-app /data/local/tmp/example_00.json 2>&1
11:55:27  INFO    : #### Read energy consumption: /data/work/juno/lisa/results/20160928_115441/energy.json
11:55:27  INFO    : #### Stop FTrace
11:55:27  INFO    : #### Save FTrace: /data/work/juno/lisa/results/20160928_115441/trace.dat
11:55:30  INFO    : #### Save platform description: /data/work/juno/lisa/results/20160928_115441/platform.json


In [ ]:
configure_target(1,0)

logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [22]:
print te.emeter.readings


{}


In [ ]:
configure_target(0,1024)

logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [ ]:
configure_target(1,1024)

logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)